$$一维谐振子变分法求解：基于高斯基函数（\textbf{Gaussian Basis Functions}）$$
1. 物理模型与哈密顿量考虑一维谐振子势场：$$V(x) = \frac{1}{2}m\omega^2x^2$$系统的哈密顿量（Hamiltonian）定义为：
$$\hat{H} = -\frac{\hbar^2}{2m} \frac{d^2}{dx^2} + \frac{1}{2}m\omega^2x^2$$

2. 基函数选择为了求解该问题，我们选取高斯型函数作为基函数（Basis Functions）。与固定宽度的做法不同，此处引入可变宽度参数 $\nu_i$ 和中心位置参数 $s_i$：
$$\varphi_i(x) = \exp\left[-\nu_i (x - s_i)^2\right]$$
其中：$\nu_i > 0$ 控制高斯函数的宽度（Width/Spread）。$s_i$ 控制高斯函数的中心位置（Center）。

3. 矩阵元计算定义两个高斯基函数的重叠积分（Overlap Integral）：
$$O_{ij} = \langle \varphi_i | \varphi_j \rangle = \int_{-\infty}^{+\infty} \varphi_i(x) \varphi_j(x) \, dx$$
积分结果为：
$$O_{ij} = \sqrt{\frac{\pi}{\nu_i+\nu_j}} \exp\left[-\frac{\nu_i \nu_j}{\nu_i+\nu_j} (s_i - s_j)^2\right]$$
哈密顿矩阵元总哈密顿矩阵元 $H_{ij}$ 由动能项 $T_{ij}$ 和势能项 $P_{ij}$ 组成：$$H_{ij} = T_{ij} + P_{ij}$$为了简化计算，采用自然单位制 ($\hbar = 1, m = 1, \omega = 1$)。在此单位制下，矩阵元的解析表达式如下：1. 势能矩阵元 ($P_{ij}$)对应算符 $\frac{1}{2}x^2$：
$$\begin{aligned}
P_{ij} &= \langle \varphi_i | \frac{1}{2}x^2 | \varphi_j \rangle \\
&= \frac{1}{2} O_{ij} \left[ \frac{1}{2(\nu_i+\nu_j)} + \left(\frac{\nu_i s_i + \nu_j s_j}{\nu_i+\nu_j}\right)^2 \right]
\end{aligned}$$

2. 动能矩阵元 ($T_{ij}$)对应算符 $-\frac{1}{2}\frac{d^2}{dx^2}$：
$$\begin{aligned}
T_{ij} &= \langle \varphi_i | -\frac{1}{2} \frac{d^2}{dx^2} | \varphi_j \rangle \\
&= O_{ij} \cdot \frac{\nu_i \nu_j}{\nu_i+\nu_j} \left[ 1 - \frac{2 \nu_i \nu_j}{\nu_i+\nu_j}(s_i - s_j)^2 \right]
\end{aligned}$$
注：此处公式已将原文中的 * 替换为标准的数学格式，并调整了分式结构以增强清晰度。
4. 广义本征值问题根据瑞利-里兹变分原理（Rayleigh-Ritz Variational Principle），求解薛定谔方程转化为求解以下的广义本征值问题（Generalized Eigenvalue Problem）：
$$\sum_{j=1}^{N} H_{ij} c_{j} = \epsilon_k \sum_{j=1}^{N} O_{ij} c_{j}$$
写成矩阵形式为：$$\mathbf{H}\mathbf{c} = \epsilon \mathbf{O}\mathbf{c}$$
其中：$\mathbf{c}$ 是展开系数向量。$\epsilon_k$ 是第 $k$ 个本征能量。
5. 优化策略本模型的改进点在于引入了高斯宽度的变化。
计算流程如下：初始化一组基函数的参数 $\{s_i, \nu_i\}$。构建矩阵 $\mathbf{H}$ 和 $\mathbf{O}$。求解广义本征值方程，得到基态能量 $\epsilon_0$（或激发态能量）。

迭代优化：通过调节参数 $s_i$ 和 $\nu_i$（例如使用梯度下降法或随机搜索算法），使 $\epsilon_k$ 最小化。当 $\epsilon_k$ 收敛到最小值时，对应的 $\epsilon_k$ 即为变分法求得的本征能量近似值，对应的波函数 $\Psi(x) = \sum c_j \varphi_j(x)$ 为本征波函数。

参考：
[1] https://zhuanlan.zhihu.com/p/596657527

In [ ]:
import numpy as np
import math

def solve_eigenproblem():
    n = 101
    
    # 参数初始化
    s = np.zeros(n)
    v = np.zeros(n)
    for i in range(n):
        s[i] = -25.0 + (i - 1) * 0.5
        v[i] = 0.5 + (i-1)*0.01          
    
    # 计算重叠矩阵O和哈密顿矩阵H
    h = np.zeros((n, n))
    o = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ss = (s[i] - s[j])**2
            # 重叠矩阵
            o[i, j] = math.sqrt(math.pi/(v[i]+v[j])) * np.exp(-(v[i]*v[j]/(v[i]+v[j])) * ss)
            # 势能矩阵元
            P = 0.5 * (1/(2*(v[i]+v[j])) + ((v[i]*s[i]+v[j]*s[j])/(v[i]+v[j]))**2) * o[i, j]
            # 动能矩阵元
            T = (v[i]*v[j]/(v[i]+v[j])) * (1 - 2*v[i]*v[j]/(v[i]+v[j]) * ss) * o[i, j]
            # 总哈密顿矩阵元
            h[i, j] = T + P
    
    # 修正：正确求解广义本征值问题 Hc = εOc
    # 使用 np.linalg.eig 求解 (H - λO)c = 0
    eigenvalues, eigenvectors = np.linalg.eig(np.linalg.solve(o, h))
    
    # 找到最小本征值的索引
    min_idx = np.argmin(eigenvalues.real)
    min_eigenvalue = eigenvalues[min_idx].real
    eigenvector_min = eigenvectors[:, min_idx]
    
    # 归一化本征向量（根据重叠矩阵）
    norm = np.sqrt(np.dot(eigenvector_min.conj(), np.dot(o, eigenvector_min)))
    if abs(norm) > 1e-10:
        eigenvector_min = eigenvector_min / norm
    
    # 找到本征向量中绝对值最大的系数对应的索引
    max_coeff_idx = np.argmax(np.abs(eigenvector_min))
    
    print(f'最小能量本征值: {min_eigenvalue}')
    print(f'对应的S值: {s[max_coeff_idx]}')
    print(f'对应的V值: {v[max_coeff_idx]}')
    
    # 找到倒数第二小本征值的索引
    sorted_indices = np.argsort(eigenvalues.real)


    idx = sorted_indices[1]
    print(f"  索引={idx}, 本征值={eigenvalues[idx].real:.6f}")
    # 获取前6个本征值对应的索引
    first_six_indices = sorted_indices[:6]
    first_six_eigenvectors = eigenvectors[:, first_six_indices]
    eigenvector_idx=eigenvectors[:, idx]
    
    return eigenvalues, eigenvectors, s, v, min_idx, eigenvector_min,idx,eigenvector_idx,first_six_indices, first_six_eigenvectors

# 运行
eigenvalues, eigenvectors, s, v, min_idx, eigenvector_min, idx,eigenvector_idx,first_six_indices, first_six_eigenvectors= solve_eigenproblem()
n=101

In [3]:
import matplotlib.pyplot as plt
import numpy as np

# 创建x坐标网格
x = np.linspace(-10, 10, 1000)

# 创建图形
fig, axes = plt.subplots(2, 3, figsize=(12, 8), dpi=800)

# 解析解函数
def analytic_wavefunction(x, n):
    if n == 0:  # 基态
        return np.pi**(-0.25) * np.exp(-x**2/2)
    elif n == 1:  # 第一激发态
        return -(1/np.sqrt(2)) * np.pi**(-0.25) * 2*x * np.exp(-x**2/2)
    elif n == 2:  # 第二激发态
        return -(1/np.sqrt(8)) * np.pi**(-0.25) * (4*x**2 - 2) * np.exp(-x**2/2)
    elif n == 3:  # 第三激发态
        return (1/np.sqrt(48)) * np.pi**(-0.25) * (8*x**3 - 12*x) * np.exp(-x**2/2)
    elif n == 4:  # 第四激发态
        return -(1/(np.sqrt(384))) * np.pi**(-0.25) * (16*x**4 - 48*x**2 + 12) * np.exp(-x**2/2)
    elif n == 5:  # 第五激发态
        return (1/(np.sqrt(3840))) * np.pi**(-0.25) * (32*x**5 - 160*x**3 + 120*x) * np.exp(-x**2/2)
    return np.zeros_like(x)        #解析公式来源于周世勋 量子力学教程

# 画前6个波函数
for n in range(6):
    ax = axes[n//3, n%3]
    wavefunction = np.zeros_like(x, dtype=complex)
    
    # 重构波函数
    for i in range(101):
        wavefunction += first_six_eigenvectors[i,n] * np.exp(-v[i] * (x - s[i])**2)
    
    # 归一化
    norm = np.sqrt(np.trapz(np.abs(wavefunction)**2, x))
    if abs(norm) > 1e-10:
        wavefunction = wavefunction / norm
    
    # 画图
    ax.plot(x, np.real(wavefunction), 'r-', label='Numerical', linewidth=2)
    ax.scatter(x[::10], analytic_wavefunction(x, n)[::10], label='Analytical',color='k',zorder=2)
    ax.set_title(f'State {n}, E={eigenvalues[first_six_indices[n]]:.6f}', fontsize=12)
    ax.set_xlabel('x', fontsize=10)
    ax.set_ylabel('ψ(x)', fontsize=10)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.set_xlim([-5, 5])

plt.tight_layout()
plt.show()